This projcet tries to determine a value for the velocity v of the ISS based on photos taken from the space station as part of Astro Pi Mission Space Lab. 
The principle is to idenify features in consecutive frames and then use the Ground Sampling Distance (the area covered by a single pixel) to work out how far the ISS has travelled between photos (d). Extracting the time the photos were taken allows a simple v = d/t calculation to be made. 

In [25]:
import cv2
import math
from exif import Image
from datetime import datetime

Extract EXIF data from photo and access the time the photo was taken. Convert this to a datetime object and calculate the delta between the two.

In [29]:
with open('cv2comp2/photo_01931.jpg', 'rb') as image_file:
    frame1 = Image(image_file)
    print(frame1.datetime)
    i1_time = datetime.strptime(frame1.datetime, '%Y:%m:%d %H:%M:%S')
with open('cv2comp2/photo_01929.jpg', 'rb') as image_file:
    frame2 = Image(image_file)
    print(frame2.datetime) 
    i2_time = datetime.strptime(frame2.datetime, '%Y:%m:%d %H:%M:%S')
time_diff=(i1_time - i2_time).seconds

2022:01:14 21:12:43
2022:01:14 21:12:25


Open two consecutive images taken from the ISS with opencv. 

In [2]:
i1 = cv2.imread('cv2comp2/photo_01931.jpg',0)
i2 = cv2.imread('cv2comp2/photo_01929.jpg',0)


There are anumber of feature detection algorithms build into opencv. Oriented FAST and Rotated Brief (ORB) is one of them.
First of all we setup an ORB and then use it to detect the key points and compute the image descriptors for both of the images.   

In [3]:
orb = cv2.ORB_create(nfeatures=500)
kp1, des1 = orb.detectAndCompute(i1, None)
kp2, des2 = orb.detectAndCompute(i2, None)



Then we use OpenCV's Brute Force (BF) matcher to match the features extracted from one image with those from the other. The results are then sorted based on distnace (not a physical distance, but how similar the features are).

In [4]:
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1,des2)
matches = sorted(matches, key=lambda x: x.distance)

Mark the features on scaled versions of the images and connect them with lines.
PRESS a key to kill the images window - DON'T CLOSE it or else Jupyter will hang on the next cell!

In [5]:
match_img = cv2.drawMatches(i1, kp1, i2, kp2, matches[:100], None)
rs = cv2.resize(match_img, (1600,600), interpolation = cv2.INTER_AREA)
cv2.imshow('m', rs)
cv2.waitKey(0)
cv2.destroyWindow('m')

Get a clearer view of the detected features in a simgle image

In [6]:
i1kp = cv2.drawKeypoints(i1,kp1,0, (0,255,0))

In [7]:
rsi1kp = cv2.resize(i1kp, (800,600), interpolation = cv2.INTER_AREA)
cv2.imshow('k', rsi1kp)
cv2.waitKey(0)
cv2.destroyWindow('k')

Create two lists to staore the x,y coordinates of the matching points from each image and populate them by iterating over all the matches

In [10]:
list_kp1 = []
list_kp2 = []
for m in matches:
    i1_idx = m.queryIdx
    i2_idx = m.trainIdx
    (x1,y1) = kp1[i1_idx].pt
    (x2,y2) = kp2[i2_idx].pt
    list_kp1.append((x1,y1))
    list_kp2.append((x2,y2))

In [16]:
print(list_kp1[0], list_kp2[0])


(2173.82421875, 779.3280639648438) (2477.9521484375, 172.80001831054688)


Two different ways of calculating the distnace between points on images. Pick one!

In [7]:
d = math.sqrt((list_kp1[0][0] - list_kp2[0][0])**2 + (list_kp2[0][1] - list_kp1[0][1])**2)6820.325056834193

In [ ]:
d = math.hypot(list_kp1[0][0] - list_kp2[0][0], list_kp1[0][1] - list_kp2[0][1])

In [10]:
d

406.403437649094

Calculate the mean distance (in pixels) from all matches

In [11]:
a = 0
l = len(matches)
#l =100
for c in range(l):
    d = math.hypot(list_kp1[c][0] - list_kp2[c][0], list_kp1[c][1] - list_kp2[c][1])
    a = a + d
    #print(d)
print(a/l)

946.6377653643358


Calculate the Ground Sampling Distnace (GSD) - how much real world distance is represented by a pixel in the image usign this online tool
https://www.3dflow.net/ground-sampling-distance-calculator/
for these images taken with HQC and 5mm lens it is: 12648 cm/pixel
Use this to calculate velocity based on time difference of images.


In [30]:
(((a/l) * 12648/100)/time_diff)

6651.7080312933995

In [25]:
(513019/4056)*100

12648.397435897437

In [ ]:
6651.7080312933995

Correct value for ISS speed is 7.66 km/s


Possible refinements or improvements:
- Use a different feature detection algorithm
- Choose a different number of feature matches to use
- Use more than 2 photos
- Check how long a photo takes to be written to disk to get a more accurate value for the time between photos. 
- Does the curvature of the Earth have an effect on the actual distance values travelled?
- Does the height of the identified feature also have an effect?
- Does the angle of motion (diagonally across the frame) have a impact that needs to be corrected for?